In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import urllib.request as urllib

## Question1: Extracting Tesla Stock Data Using yfinance

# Tesla Stock data
## Extraction

In [2]:
tesla_html = yf.Ticker('TSLA')

In [3]:
tesla_data = tesla_html.history(period='max')

In [4]:
tesla_data.reset_index(inplace = True)

In [5]:
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2 - Extracting Tesla Revenue Data Using Webscraping

In [6]:
html_data = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
                        ).text

In [7]:
html_parse = bs(html_data, 'html5lib')

In [8]:
tesla_revenue = pd.read_html('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm')

In [9]:
len(tesla_revenue)

6

In [10]:
tesla_revenue = tesla_revenue[1]
tesla_revenue.head()

,Tesla Quarterly Revenue (Millions of US $),Tesla Quarterly Revenue (Millions of US $).1
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


In [11]:
tesla_revenue.columns[0]

'Tesla Quarterly Revenue (Millions of US $)'

In [12]:
tesla_revenue = tesla_revenue.rename(columns={tesla_revenue.columns[0]: 'Date', tesla_revenue.columns[1]: 'Revenue'})

In [13]:
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex = True)

In [14]:
tesla_revenue.dropna(inplace= True)


In [15]:
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [16]:
tesla_revenue.reset_index(inplace = True)


In [17]:
tesla_revenue = tesla_revenue.drop('index', axis = 1)

In [18]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
51,2009-09-30,46
52,2009-06-30,27


# GameStop Stock data
## Extraction

## Question 3 - Extracting GameStop Stock Data Using yfinance

In [19]:
gme = yf.Ticker('GME')
gme_data = gme.history(period = 'max')


In [20]:
gme_data.reset_index(inplace = True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603296,1.662209,6892800,0.0,0.0


## Question 4 - Extracting GameStop Revenue Data Using Webscraping

In [21]:
html_data = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
).text
parse_data = bs(html_data, 'html5lib')

In [22]:
pandas_html_data = pd.read_html(str(parse_data))

In [23]:
len(pandas_html_data)

6

In [24]:
pandas_html_data = pandas_html_data[1]
pandas_html_data.columns

Index(['GameStop Quarterly Revenue (Millions of US $)', 'GameStop Quarterly Revenue (Millions of US $).1'], dtype='object')

In [25]:
pandas_html_data = pandas_html_data.rename(columns={pandas_html_data.columns[0]: 'Date', pandas_html_data.columns[1]: 'Revenue'})

In [26]:
pandas_html_data['Revenue'] = pandas_html_data['Revenue'].str.replace(',|\$','', regex = True)
pandas_html_data.dropna(inplace = True)
pandas_html_data = pandas_html_data[pandas_html_data['Revenue'] != '']


In [27]:
gme_revenue = pandas_html_data
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


In [28]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [32]:
import plotly.io as pio

pio.renderers.default = 'iframe'

## Question 5 - Tesla Stock and Revenue Dashboard 

In [33]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## Question 6 - GameStop Stock and Revenue Dashboard

In [34]:
make_graph(gme_data, gme_revenue, 'GameStop')